In [ ]:
import ast
import functools
import inspect
import os
import sys

from IPython.core.magic import register_cell_magic, register_line_magic
from IPython import get_ipython


class TraceState(object):
    def __init__(self):
        self.call_depth = 0
        self.code_lines = {}
        self.stack = []
        self.source = None
        self.last_event = None
        self.last_line = None
        self.last_line_node = None
        self.last_frame = None
        self.last_lineno = None

# TODO: maybe frame, scope, indentation, etc
class CodeLine(object):
    def __init__(self, text, ast_node, lineno, call_depth):
        self.text = text
        self.ast_node = ast_node
        self.lineno = lineno
        self.call_depth = call_depth
        self.extra_dependencies = set()
    
    def compute_rval_dependencies(self):
        return get_all_rval_names(self.ast_node) | self.extra_dependencies

def tracefunc(frame, event, arg, state):    
    # this is a bit of a hack to get the class out of the locals
    # - it relies on 'self' being used... normally a safe assumption!
    try:
        class_name = frame.f_locals['self'].__class__.__name__ 
    except (KeyError, AttributeError):
        class_name = "No Class"
    
    # notebook filenames appear as 'ipython-input...'
    if 'ipython-input' not in frame.f_code.co_filename:
        return
    
    tracer = functools.partial(tracefunc, state=state)
    
    if state.source is None:
        state.source = inspect.getsource(frame).split('\n')
    line = state.source[frame.f_lineno-1]

    old_depth = state.call_depth
    
    # IPython quirk -- every line in outer scope apparently wrapped in lambda
    # We want to skip the outer 'call' and 'return' for these
    if event == 'call':
        state.call_depth += 1
        if old_depth == 0:
            return tracer
    
    if event == 'return':
        state.call_depth -= 1
        if old_depth <= 1:
            return tracer
    
    to_parse = line = line.strip()
    print(frame.f_lineno, state.call_depth, event, line)
    if to_parse[-1] == ':':
        to_parse += '\n    pass'
    node = ast.parse(to_parse).body[0]
    code_line = state.code_lines.get(
        frame.f_lineno, CodeLine(line, node, frame.f_lineno, state.call_depth)
    )
    state.code_lines[frame.f_lineno] = code_line
    
    if event == 'line':
        state.last_line = code_line
    if event == 'call':
        assert state.last_event == 'line'
        state.stack.append(state.last_line)
    if event == 'return':
        ret_line = state.stack.pop()
        ret_line.extra_dependencies |= code_line.compute_rval_dependencies()
        print('{} @@returning to@@ {}'.format(code_line.text, ret_line.text))
        print('dependencies: %s' % ret_line.compute_rval_dependencies())
    state.last_event = event
    state.last_frame = frame
    state.last_lineno = frame.f_lineno
    return tracer

In [ ]:
class GetAllRvalNames(ast.NodeVisitor):
    def __init__(self):
        self.name_set = set()

    def __call__(self, node):
        self.visit(node)
        return self.name_set

    def visit_Name(self, node):
        self.name_set.add(node.id)
        
    def visit_Assign(self, node):
        # skip node.targets
        self.visit(node.value)
    
    def visit_AugAssign(self, node):
        # skip node.target
        self.visit(node.value)
    
    def visit_For(self, node):
        # skip node.target (gets bound to node.iter)
        # skip body too -- will have dummy since this visitor works line-by-line
        self.visit(node.iter)
    
    def visit_Lambda(self, node):
        # remove node.arguments
        self.visit(node.body)
        old = self.name_set
        self.name_set = set()
        self.visit(node.args)
        self.name_set = old - self.name_set
    
    def visit_arg(self, node):
        self.name_set.add(node.arg)


def get_all_rval_names(node: ast.AST):
    return GetAllRvalNames()(node)

In [ ]:
@register_cell_magic
def tracecell(_, cell):
    state = TraceState()
    sys.settrace(functools.partial(tracefunc, state=state))
    get_ipython().run_cell(cell)
    sys.settrace(None)

In [ ]:
%%tracecell
y = 3 + 5
def f(x):
    def g(y):
        a = y + 5
        return a * x
    foo = lambda z: g(z) * 3
    return foo(4)

asdf = f(3)
print(asdf)

In [ ]:
x = 11
def h():
    x = 10
    def f():
        def g():
            print(x)
            print(inspect.currentframe().f_locals.get('x', 'not found'))
            print(inspect.currentframe().f_globals.get('x', 'not found'))
        return g
    return f
h()()()